In this notebook we give a proof of concept of unitary compiling using TFQ. 

In [3]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 
import tensorflow as tf
from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
import matplotlib.pyplot as plt 
import tensorflow_quantum as tfq
import cirq
from utilities.compiling import *
from utilities.misc import *
from utilities.simplifier import Simplifier
from utilities.discrimination import *
from utilities.idinserter import IdInserter
from utilities.evaluator import Evaluator
from utilities.gate_killer import GateKiller
from ast import literal_eval
import tensorflow_quantum as tfq

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
etas = [0.2,1.]

translator = CirqTranslater(3, untouchable_blocks = [1], discard_qubits=[2])
simplifier = Simplifier(translator)
inserter = IdInserter(translator.n_qubits, untouchable_blocks=translator.untouchable_blocks, untouchable_qubits = [2])
killer = GateKiller(translator, mode="discrimination", params=etas)
minimizer = Minimizer(translator, mode="discrimination", params=etas)

args_evaluator = {"n_qubits":translator.n_qubits, "problem":"acd","params":etas}
evaluator = Evaluator(args=args_evaluator, lower_bound_cost=minimizer.lower_bound_cost, nrun=0, stopping_criteria=1e-3)



In [6]:
evaluator.load_dicts_and_displaying(evaluator.identifier)

In [10]:
[circuit_db, cost, lowest_cost, lower_bound, acceptance_percentage, operation, history] = evaluator.evolution[len(evaluator.evolution.keys())-1]

In [15]:
circuit, circuit_db = translator.give_circuit(circuit_db, unresolved = False)

In [16]:
circuit

(0, 0): ───Rx(π)────────H─────────────X────────────────X───H────────────────X───Rx(-π)───
                                      │                │                    │
(0, 1): ───Rx(0.114π)───Rx(-0.114π)───┼────────────────┼────────────────────┼────────────
                                      │                │                    │
(0, 2): ───H──────────────────────────@───Ry(0.318π)───@───H───Ry(0.318π)───@────────────